# VAZHI SFT v3.2 - Fixed Training

**FIXES from v3.1 failure:**
1. **Data format consistency** - Only uses ChatML-formatted samples (no raw text mixing)
2. **Pinned versions** - Avoids API drift issues
3. **Single GPU forced** - Prevents cuda:1 vs cuda:0 errors
4. **fp16 on T4** - Not bf16 (T4 doesn't support bf16 well)
5. **4-bit QLoRA** - More stable/memory efficient on Kaggle

**Root cause of v3.1 failure:**
Mixed raw text (Sangam poetry, Thirukkural verses) with ChatML-formatted Q&A pairs.
This caused the model to output "systemsystemsystem..." garbage.

**Per GPT5.2 recommendations.**

## 1. Setup & Dependencies

**IMPORTANT:** After running the install cell, **RESTART the Kaggle session** (Kernel → Restart Session)

In [ ]:
# Install dependencies - Updated versions for Qwen3 support
# NOTE: transformers 4.46.3 does NOT support Qwen3 (too old)
# Using newer versions that support Qwen3 while keeping other libs stable
# 
# IMPORTANT: After running this cell, RESTART the Kaggle session!

!pip -q install -U \
  "transformers>=4.51.0" \
  "accelerate>=0.34.2" \
  "peft>=0.12.0" \
  "trl>=0.12.0" \
  "bitsandbytes>=0.43.3" \
  "datasets>=2.21.0" \
  "huggingface_hub>=0.24.7"

# Verify Qwen3 is supported
import transformers
print(f"✅ Transformers version: {transformers.__version__}")
print("✅ Dependencies installed")
print("⚠️ IMPORTANT: Restart the Kaggle session now (Kernel → Restart Session)")

In [ ]:
# ============================================================================
# CRITICAL: Force single GPU BEFORE importing torch/transformers
# Per GPT5.2: This prevents "cuda:1 vs cuda:0" device mismatch errors
# Must be at the VERY TOP before any other imports
# ============================================================================
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import random
import re
from collections import defaultdict
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm
from huggingface_hub import login, HfApi

# Config
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# HuggingFace repos
EXISTING_DATASET = "CryptoYogi/vazhi-tamil-v05"
BALANCED_DATASET = "CryptoYogi/vazhi-tamil-sft-v3_2"  # New version

# System prompt
SYSTEM_PROMPT = "நீங்கள் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர். தமிழில் தெளிவாகவும் உதவியாகவும் பதிலளியுங்கள். தெரியாவிட்டால் \"தெரியவில்லை\" என்று சொல்லுங்கள்."

print("✅ Configuration loaded")
print(f"   CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES', 'not set')}")
print(f"   Source: {EXISTING_DATASET}")
print(f"   Target: {BALANCED_DATASET}")

In [ ]:
# Login to HuggingFace
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
hf_token = secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("✅ Logged in to HuggingFace")

## 2. Helper Functions

In [ ]:
# ============================================================================
# CHECK: Skip data preparation if balanced dataset already exists
# ============================================================================
SKIP_DATA_PREP = False

try:
    from huggingface_hub import dataset_info
    info = dataset_info(BALANCED_DATASET)
    print(f"✅ Dataset {BALANCED_DATASET} already exists on HuggingFace!")
    print(f"   Created: {info.created_at}")
    print(f"   Downloads: {info.downloads}")
    print(f"\n🚀 SKIPPING data extraction/preparation - will load directly for training")
    SKIP_DATA_PREP = True
except Exception as e:
    print(f"📝 Dataset {BALANCED_DATASET} not found. Will create it.")
    print(f"   (This is expected on first run)")
    SKIP_DATA_PREP = False

## 3. Extract Diverse QA from IndicAlign

IndicAlign contains Tamil translations in the `tam_Taml` field (translated using IndicTrans2 by AI4Bharat).

In [ ]:
def extract_from_indicaling(config_name, max_samples):
    """Extract Tamil samples from IndicAlign config.
    
    IndicAlign structure:
    - tam_Taml is a list of length 1
    - tam_Taml[0] is a list of conversation turns [user, assistant, ...]
    - tam_Taml[0][0] = user message (Tamil)
    - tam_Taml[0][1] = assistant message (Tamil)
    """
    print(f"\n📚 Loading {config_name}...")
    try:
        ds = load_dataset("ai4bharat/indic-align", config_name, split="train", streaming=True)
    except Exception as e:
        print(f"   ⚠️ Error: {e}")
        return []
    
    samples = []
    seen = set()
    skipped_non_tamil = 0
    skipped_short = 0
    skipped_format = 0
    
    for item in tqdm(ds, desc=config_name, total=max_samples*5):
        if len(samples) >= max_samples:
            break
        
        # tam_Taml is a nested list: [[user_msg, assistant_msg, ...]]
        tamil = item.get('tam_Taml', [])
        if not tamil:
            continue
        
        # FIXED: Access nested list structure correctly
        if isinstance(tamil, list) and len(tamil) > 0:
            turns = tamil[0]  # Get the inner list
            if isinstance(turns, list) and len(turns) >= 2:
                user_msg = clean_text(str(turns[0]))
                assistant_msg = clean_text(str(turns[1]))
            else:
                skipped_format += 1
                continue
        else:
            skipped_format += 1
            continue
        
        # Verify it's actually Tamil
        if not is_good_tamil_sample(user_msg):
            skipped_non_tamil += 1
            continue
        if not is_good_tamil_sample(assistant_msg):
            skipped_short += 1
            continue
        
        # Dedup
        key = user_msg[:100]
        if key in seen:
            continue
        seen.add(key)
        
        samples.append({
            "instruction": user_msg,
            "output": assistant_msg,
            "source": config_name
        })
    
    print(f"   ✅ Extracted {len(samples)} Tamil samples")
    print(f"   ⏭️ Skipped: {skipped_non_tamil} (not Tamil), {skipped_short} (too short), {skipped_format} (wrong format)")
    
    if samples:
        print(f"   📝 Sample verification:")
        print(f"      User: {samples[0]['instruction'][:80]}...")
        print(f"      Asst: {samples[0]['output'][:80]}...")
    
    return samples

In [ ]:
# Extract from IndicAlign (SKIP if dataset already exists)
if not SKIP_DATA_PREP:
    print("🚀 Extracting diverse QA from IndicAlign...")
    print("   (tam_Taml field = Already-translated Tamil via IndicTrans2)")

    diverse_samples = []
    diverse_samples.extend(extract_from_indicaling("Dolly_T", 300))
    diverse_samples.extend(extract_from_indicaling("WikiHow", 250))
    diverse_samples.extend(extract_from_indicaling("Wiki_Conv", 300))
    diverse_samples.extend(extract_from_indicaling("OpenAssistant_T", 200))

    print(f"\n📊 Total extracted from IndicAlign: {len(diverse_samples)}")
else:
    print("⏭️ Skipping IndicAlign extraction (dataset already exists)")

In [ ]:
# Verify Tamil content distribution (SKIP if dataset already exists)
if not SKIP_DATA_PREP:
    tamil_char_pcts = []
    for s in diverse_samples[:100]:
        text = s['instruction'] + s['output']
        pct = 100 * count_tamil_chars(text) / len(text) if text else 0
        tamil_char_pcts.append(pct)

    avg_tamil_pct = sum(tamil_char_pcts) / len(tamil_char_pcts) if tamil_char_pcts else 0
    print(f"📈 Average Tamil character % in samples: {avg_tamil_pct:.1f}%")

    if avg_tamil_pct < 40:
        print("⚠️ Warning: Tamil content seems low. Check extraction logic.")
    else:
        print("✅ Good Tamil content ratio!")
else:
    print("⏭️ Skipping Tamil verification (dataset already exists)")

## 4. Add Manual Samples (Short Answers + Behavior)

In [ ]:
# Manual samples (SKIP if dataset already exists)
if not SKIP_DATA_PREP:
    manual_samples = [
        # Geography
        {"instruction": "தமிழ்நாட்டின் தலைநகரம் என்ன?", "output": "சென்னை.", "source": "manual"},
        {"instruction": "இந்தியாவின் தலைநகரம் எது?", "output": "புது தில்லி.", "source": "manual"},
        {"instruction": "உலகின் மிகப்பெரிய நாடு எது?", "output": "ரஷ்யா (பரப்பளவில்).", "source": "manual"},
        {"instruction": "தமிழ்நாட்டின் மாவட்டங்கள் எத்தனை?", "output": "38 மாவட்டங்கள்.", "source": "manual"},
        {"instruction": "காவிரி நதி எந்த மாநிலங்களில் பாய்கிறது?", "output": "கர்நாடகா மற்றும் தமிழ்நாடு.", "source": "manual"},
        {"instruction": "மதுரை எந்த நதிக்கரையில் உள்ளது?", "output": "வைகை நதிக்கரையில்.", "source": "manual"},
        {"instruction": "கங்கை நதி எங்கு உற்பத்தியாகிறது?", "output": "இமயமலையில் உள்ள கங்கோத்ரி பனிப்பாறையில்.", "source": "manual"},
        {"instruction": "இந்தியாவின் மக்கள்தொகை அதிகமான மாநிலம் எது?", "output": "உத்தரப் பிரதேசம்.", "source": "manual"},
        
        # Basic facts
        {"instruction": "சூரியன் எந்த திசையில் உதிக்கும்?", "output": "கிழக்கு திசையில்.", "source": "manual"},
        {"instruction": "ஒரு வாரத்தில் எத்தனை நாட்கள்?", "output": "ஏழு நாட்கள்.", "source": "manual"},
        {"instruction": "ஒரு வருடத்தில் எத்தனை மாதங்கள்?", "output": "12 மாதங்கள்.", "source": "manual"},
        {"instruction": "தண்ணீரின் கொதிநிலை என்ன?", "output": "100 டிகிரி செல்சியஸ்.", "source": "manual"},
        {"instruction": "2+2 என்ன?", "output": "4.", "source": "manual"},
        {"instruction": "10 x 10 என்ன?", "output": "100.", "source": "manual"},
        {"instruction": "100-ஐ 4-ஆல் வகுத்தால்?", "output": "25.", "source": "manual"},
        {"instruction": "பூமி சூரியனை சுற்ற எத்தனை நாட்கள் ஆகும்?", "output": "365 நாட்கள் (ஒரு வருடம்).", "source": "manual"},
        
        # Tamil culture (non-Thirukkural)
        {"instruction": "பொங்கல் எப்போது கொண்டாடப்படுகிறது?", "output": "தை மாதம் முதல் நாள் (ஜனவரி 14 அல்லது 15).", "source": "manual"},
        {"instruction": "தமிழ் எழுத்துக்கள் எத்தனை?", "output": "247 எழுத்துக்கள் (12 உயிர் + 18 மெய் + 216 உயிர்மெய் + 1 ஆய்தம்).", "source": "manual"},
        {"instruction": "சிலப்பதிகாரத்தை எழுதியவர் யார்?", "output": "இளங்கோவடிகள்.", "source": "manual"},
        {"instruction": "பாரதியார் எந்த ஊரில் பிறந்தார்?", "output": "எட்டயபுரம்.", "source": "manual"},
        {"instruction": "தமிழ் தினம் எப்போது?", "output": "ஜனவரி 9.", "source": "manual"},
        {"instruction": "தமிழ்நாட்டின் அலுவல் மொழி என்ன?", "output": "தமிழ்.", "source": "manual"},
        
        # Science
        {"instruction": "மனித உடலில் எத்தனை எலும்புகள் உள்ளன?", "output": "206 எலும்புகள்.", "source": "manual"},
        {"instruction": "H2O என்பது என்ன?", "output": "தண்ணீர் (நீர்).", "source": "manual"},
        {"instruction": "பூமியின் ஒரே இயற்கை துணைக்கோள் எது?", "output": "நிலவு (சந்திரன்).", "source": "manual"},
        {"instruction": "சூரிய குடும்பத்தில் எத்தனை கோள்கள்?", "output": "எட்டு கோள்கள்.", "source": "manual"},
        {"instruction": "ஒளியின் வேகம் என்ன?", "output": "வினாடிக்கு சுமார் 3 லட்சம் கிலோமீட்டர்.", "source": "manual"},
        
        # Everyday Tamil
        {"instruction": "நன்றி என்றால் ஆங்கிலத்தில் என்ன?", "output": "Thank you.", "source": "manual"},
        {"instruction": "Good morning தமிழில் என்ன?", "output": "காலை வணக்கம்.", "source": "manual"},
        {"instruction": "வணக்கம் என்றால் என்ன?", "output": "வணக்கம் என்பது தமிழில் பயன்படுத்தப்படும் வாழ்த்துச் சொல், Hello என்ற பொருளில்.", "source": "manual"},
        {"instruction": "ஆம் என்றால் ஆங்கிலத்தில்?", "output": "Yes.", "source": "manual"},
        {"instruction": "இல்லை என்றால் ஆங்கிலத்தில்?", "output": "No.", "source": "manual"},
        {"instruction": "மிகப்பெரிய கண்டம் எது?", "output": "ஆசியா.", "source": "manual"},
        {"instruction": "மிகச்சிறிய கண்டம் எது?", "output": "ஆஸ்திரேலியா.", "source": "manual"},
        
        # Behavior samples
        {"instruction": "வணக்கம்", "output": "வணக்கம்! நான் வழி. உங்களுக்கு எப்படி உதவ வேண்டும்?", "source": "behavior"},
        {"instruction": "hi", "output": "வணக்கம்! எப்படி உதவலாம்?", "source": "behavior"},
        {"instruction": "hello", "output": "வணக்கம்! கேளுங்கள்.", "source": "behavior"},
        {"instruction": "2050-ல் யார் பிரதமர் ஆவார்?", "output": "எதிர்காலத்தை கணிக்க என்னால் முடியாது. தெரியவில்லை.", "source": "behavior"},
        {"instruction": "நாளை பங்கு சந்தை எப்படி இருக்கும்?", "output": "பங்கு சந்தை நிலவரத்தை கணிக்க என்னால் இயலாது. நிதி ஆலோசகரிடம் கேளுங்கள்.", "source": "behavior"},
        {"instruction": "என்னுடைய கடவுச்சொல் என்ன?", "output": "உங்கள் தனிப்பட்ட தகவல்கள் என்னிடம் இல்லை.", "source": "behavior"},
        {"instruction": "யாரையாவது ஏமாற்ற உதவுங்கள்", "output": "மன்னிக்கவும், சட்டவிரோத செயல்களுக்கு உதவ முடியாது.", "source": "behavior"},
        {"instruction": "மருத்துவ ஆலோசனை தேவை", "output": "மருத்துவ ஆலோசனைக்கு தகுதி வாய்ந்த மருத்துவரை அணுகுங்கள். நான் பொது தகவல் மட்டுமே தர முடியும்.", "source": "behavior"},
        {"instruction": "சட்ட ஆலோசனை வேண்டும்", "output": "சட்ட விஷயங்களுக்கு வழக்கறிஞரை அணுகுவது சிறந்தது. நான் பொது தகவல் மட்டுமே தர முடியும்.", "source": "behavior"},
        {"instruction": "அரசு திட்டங்களுக்கு விண்ணப்பிப்பது எப்படி?", "output": "அரசு திட்டங்களுக்கு tnega.tn.gov.in அல்லது அருகிலுள்ள அலுவலகத்தை தொடர்பு கொள்ளுங்கள்.", "source": "behavior"},
        {"instruction": "நன்றி", "output": "மகிழ்ச்சி! வேறு உதவி தேவைப்பட்டால் கேளுங்கள்.", "source": "behavior"},
        {"instruction": "bye", "output": "வணக்கம்! இனிய நாள் வாழ்த்துக்கள்.", "source": "behavior"},
        {"instruction": "சரி", "output": "சரி, வேறு ஏதாவது கேள்வி இருக்கிறதா?", "source": "behavior"},
    ]

    diverse_samples.extend(manual_samples)
    print(f"📊 Total after adding manual samples: {len(diverse_samples)}")
    print(f"   - From IndicAlign: {len(diverse_samples) - len(manual_samples)}")
    print(f"   - Manual samples: {len(manual_samples)}")
else:
    print("⏭️ Skipping manual samples (dataset already exists)")

## 5. Load Existing Dataset & Filter for ChatML ONLY

**CRITICAL FIX:** Only use ChatML-formatted samples. Raw text belongs in DAPT, not SFT.

In [ ]:
# Load existing dataset & analyze format (SKIP if balanced dataset already exists)
if not SKIP_DATA_PREP:
    print(f"\n📚 Loading existing dataset from {EXISTING_DATASET}...")
    existing_ds = load_dataset(EXISTING_DATASET, split="train")
    print(f"   Loaded {len(existing_ds)} samples")

    # Analyze format distribution BEFORE filtering
    chatml_count = 0
    raw_count = 0
    for item in tqdm(existing_ds, desc="Analyzing formats"):
        text = item.get('text', '')
        if is_chatml_formatted(text):
            chatml_count += 1
        else:
            raw_count += 1

    print(f"\n📊 Existing dataset format analysis:")
    print(f"   ChatML formatted: {chatml_count} ({100*chatml_count/len(existing_ds):.1f}%)")
    print(f"   Raw text: {raw_count} ({100*raw_count/len(existing_ds):.1f}%)")
    print(f"")
    print(f"   ⚠️ Raw text samples will be EXCLUDED from SFT training")
    print(f"   📝 They belong in Micro-DAPT stage, not SFT")
else:
    print("⏭️ Skipping existing dataset analysis (balanced dataset already exists)")

In [ ]:
# ============================================================================
# CRITICAL FIX: Filter existing dataset for ChatML ONLY
# Raw text and ChatML mixed = "systemsystemsystem..." garbage
# ============================================================================

if not SKIP_DATA_PREP:
    # Filter existing dataset - ONLY keep ChatML formatted samples
    existing_chatml_samples = []
    existing_kural_chatml = []
    existing_other_chatml = []

    for item in tqdm(existing_ds, desc="Filtering ChatML"):
        text = item.get('text', '')
        if is_chatml_formatted(text):
            if is_kural(text):
                existing_kural_chatml.append({"text": text})
            else:
                existing_other_chatml.append({"text": text})

    print(f"\n📊 ChatML samples from existing dataset:")
    print(f"   Kural (ChatML): {len(existing_kural_chatml)}")
    print(f"   Other (ChatML): {len(existing_other_chatml)}")
    print(f"   Total usable: {len(existing_kural_chatml) + len(existing_other_chatml)}")
else:
    print("⏭️ Skipping ChatML filtering (balanced dataset already exists)")

## 6. Downsample Thirukkural & Create Balanced Dataset

In [ ]:
# Downsample Thirukkural to ~25% of non-Thirukkural samples (SKIP if balanced dataset exists)
if not SKIP_DATA_PREP:
    total_other = len(existing_other_chatml)
    target_kural_pct = 0.25
    target_kural_count = int(target_kural_pct * total_other / (1 - target_kural_pct))

    print(f"\n🎯 Downsampling Thirukkural:")
    print(f"   Current ChatML Kural: {len(existing_kural_chatml)}")
    print(f"   Target: {target_kural_count} ({100*target_kural_pct:.0f}%)")

    # Randomly sample (seeded for reproducibility)
    if len(existing_kural_chatml) > target_kural_count:
        downsampled_kural = random.sample(existing_kural_chatml, target_kural_count)
    else:
        downsampled_kural = existing_kural_chatml
    print(f"   Downsampled: {len(downsampled_kural)}")
else:
    print("⏭️ Skipping Thirukkural downsampling (balanced dataset already exists)")

In [ ]:
# Combine all samples and verify format (SKIP if balanced dataset exists)
if not SKIP_DATA_PREP:
    # Convert diverse QA to ChatML format
    diverse_formatted = [{"text": to_chatml(s["instruction"], s["output"])} for s in diverse_samples]

    # Combine ALL samples - all must be ChatML formatted
    final_samples = []
    final_samples.extend(downsampled_kural)      # ChatML Kural
    final_samples.extend(existing_other_chatml)  # ChatML Other
    final_samples.extend(diverse_formatted)       # ChatML Diverse

    # Shuffle (seeded for reproducibility)
    random.shuffle(final_samples)

    print(f"\n📊 Final SFT dataset (ChatML ONLY):")
    print(f"   Downsampled Kural: {len(downsampled_kural)}")
    print(f"   Other (ChatML): {len(existing_other_chatml)}")
    print(f"   Diverse QA (new): {len(diverse_formatted)}")
    print(f"   ───────────────────")
    print(f"   Total: {len(final_samples)}")

    # CRITICAL: Verify 100% ChatML
    chatml_count = sum(1 for s in final_samples if is_chatml_formatted(s["text"]))
    print(f"\n📈 ChatML format %: {100*chatml_count/len(final_samples):.1f}% (MUST be 100%)")

    if chatml_count != len(final_samples):
        print("❌ ERROR: Not all samples are ChatML formatted! This will cause training failure.")
        raise ValueError("Data format inconsistency detected")
    else:
        print("✅ All samples are ChatML formatted - safe to train")

    # Verify Kural distribution
    final_kural = sum(1 for s in final_samples if is_kural(s["text"]))
    print(f"📈 Final Thirukkural %: {100*final_kural/len(final_samples):.1f}%")
else:
    print("⏭️ Skipping sample combination (balanced dataset already exists)")

## 7. Save & Upload to HuggingFace

In [ ]:
# Save locally and split train/val (SKIP if balanced dataset exists)
if not SKIP_DATA_PREP:
    # Create output directory
    os.makedirs("/kaggle/working/balanced_sft", exist_ok=True)

    # Split 95/5 train/val
    split_idx = int(0.95 * len(final_samples))
    train_samples = final_samples[:split_idx]
    val_samples = final_samples[split_idx:]

    # Save locally
    with open("/kaggle/working/balanced_sft/train.jsonl", 'w') as f:
        for s in train_samples:
            f.write(json.dumps(s, ensure_ascii=False) + '\n')

    with open("/kaggle/working/balanced_sft/val.jsonl", 'w') as f:
        for s in val_samples:
            f.write(json.dumps(s, ensure_ascii=False) + '\n')

    print(f"💾 Saved locally:")
    print(f"   Train: {len(train_samples)} samples")
    print(f"   Val: {len(val_samples)} samples")
else:
    print("⏭️ Skipping local save (balanced dataset already exists)")

In [ ]:
# Upload to HuggingFace (SKIP if balanced dataset exists)
if not SKIP_DATA_PREP:
    api = HfApi()

    # Create dataset repo (per GPT5.2: ensure repo exists before pushing)
    try:
        api.create_repo(BALANCED_DATASET, repo_type="dataset", exist_ok=True)
        print(f"✅ Created/verified repo: {BALANCED_DATASET}")
    except Exception as e:
        print(f"⚠️ Repo creation: {e}")

    # Upload files
    api.upload_file(
        path_or_fileobj="/kaggle/working/balanced_sft/train.jsonl",
        path_in_repo="train.jsonl",
        repo_id=BALANCED_DATASET,
        repo_type="dataset"
    )
    api.upload_file(
        path_or_fileobj="/kaggle/working/balanced_sft/val.jsonl",
        path_in_repo="val.jsonl",
        repo_id=BALANCED_DATASET,
        repo_type="dataset"
    )

    print(f"\n✅ Uploaded to: https://huggingface.co/datasets/{BALANCED_DATASET}")
else:
    print("⏭️ Skipping HuggingFace upload (balanced dataset already exists)")
    print(f"   Will load directly from: https://huggingface.co/datasets/{BALANCED_DATASET}")

## 8. Load Balanced Dataset for Training

In [ ]:
# Load the balanced dataset for training
print(f"\n📚 Loading balanced dataset for training...")
balanced_ds = load_dataset(BALANCED_DATASET, split="train")
print(f"✅ Loaded {len(balanced_ds)} balanced samples")

# Show sample - verify it's ChatML formatted
print(f"\n📝 Sample (should show ChatML tags):")
sample_text = balanced_ds[0]['text'][:400]
print(sample_text + "...")

if "<|im_start|>" in sample_text:
    print("\n✅ Sample is ChatML formatted")
else:
    print("\n❌ ERROR: Sample is NOT ChatML formatted!")

---

## 9. SFT Training Setup

Now we train Qwen3-0.6B on the balanced dataset.

**Per GPT5.2 recommendations:**
- 4-bit QLoRA (more stable/memory efficient)
- fp16 compute dtype (T4 doesn't support bf16 well)
- Pinned library versions
- Single GPU forced

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# Try importing SFTConfig (newer TRL versions)
try:
    from trl import SFTConfig
    print("✅ Using TRL with SFTConfig (newer API)")
except ImportError:
    SFTConfig = None
    print("⚠️ Using TRL with TrainingArguments (older API)")

# Model config
BASE_MODEL = "Qwen/Qwen3-0.6B"
OUTPUT_MODEL = "CryptoYogi/vazhi-qwen3-v3_2"

print(f"🤖 Base model: {BASE_MODEL}")
print(f"📤 Output: {OUTPUT_MODEL}")

In [ ]:
# Load model and tokenizer with 4-bit quantization
# 4-bit + LoRA is more stable/fast on Kaggle
print("\n📥 Loading model and tokenizer...")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
# IMPORTANT: Do NOT modify tokenizer.pad_token = eos_token
# Per TRAINING_LOG lesson #9: This causes "OrderedVocab holes" and corrupts the model
tokenizer.padding_side = "right"

# 4-bit quantization config (Kaggle-friendly)
# Use float16 compute dtype - P100/T4 don't support bf16 (requires Ampere+)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model with 4-bit quantization
# CRITICAL: Force torch_dtype=float16 - Qwen3 defaults to bf16 which P100 doesn't support
# bf16 requires Ampere architecture (A100, RTX 30xx) or newer
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,  # Force fp16 (P100/T4 don't support bf16)
    device_map={"":0},  # Force single GPU (prevents cuda:1 vs cuda:0 errors)
    trust_remote_code=True
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Align model config with tokenizer (don't modify tokenizer)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

# Disable cache for gradient checkpointing compatibility
model.config.use_cache = False

print(f"✅ Model loaded: {model.num_parameters():,} parameters (4-bit quantized)")
print(f"   torch_dtype: float16 (P100 compatible)")
print(f"   pad_token_id: {tokenizer.pad_token_id}")
print(f"   eos_token_id: {tokenizer.eos_token_id}")

In [ ]:
# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Training arguments (TRL 0.11.4 compatible)
training_args = TrainingArguments(
    output_dir="/kaggle/working/vazhi-v3_2",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # Effective batch = 16
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=25,
    save_steps=200,
    save_total_limit=2,
    fp16=True,  # T4 compatible (not bf16)
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    optim="paged_adamw_8bit",
    report_to="none",
    remove_unused_columns=False,
)

print("✅ Training arguments configured")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")

In [ ]:
# Create trainer - compatible with TRL 0.12+
try:
    from trl import SFTConfig
    
    sft_config = SFTConfig(
        output_dir="/kaggle/working/vazhi-v3_2",
        num_train_epochs=2,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=1e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        logging_steps=25,
        save_steps=200,
        save_total_limit=2,
        fp16=True,
        bf16=False,  # Explicitly disable bf16 (P100 doesn't support it)
        gradient_checkpointing=True,
        max_grad_norm=1.0,
        optim="paged_adamw_8bit",
        report_to="none",
        dataset_text_field="text",
        max_length=512,  # Changed from max_seq_length (TRL API change)
        packing=False,
    )
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=balanced_ds,
        args=sft_config,
        processing_class=tokenizer,
    )
    print("✅ Trainer initialized (SFTConfig API)")
    
except ImportError:
    # Fall back to old API
    trainer = SFTTrainer(
        model=model,
        train_dataset=balanced_ds,
        args=training_args,
        tokenizer=tokenizer,
        dataset_text_field="text",
        max_seq_length=512,
        packing=False,
    )
    print("✅ Trainer initialized (TrainingArguments API)")

In [ ]:
# Train!
print("\n🚀 Starting training...")
trainer.train()
print("\n✅ Training complete!")

In [ ]:
# Save and push to HuggingFace
print("\n💾 Saving model...")
trainer.save_model("/kaggle/working/vazhi-v3_2-final")

# Merge LoRA weights
print("\n🔀 Merging LoRA weights...")
merged_model = model.merge_and_unload()

# Ensure repo exists before pushing
api = HfApi()
try:
    api.create_repo(OUTPUT_MODEL, exist_ok=True)
    print(f"✅ Created/verified repo: {OUTPUT_MODEL}")
except Exception as e:
    print(f"⚠️ Repo creation: {e}")

# Push to HuggingFace
print(f"\n📤 Pushing to {OUTPUT_MODEL}...")
merged_model.push_to_hub(OUTPUT_MODEL, private=False)
tokenizer.push_to_hub(OUTPUT_MODEL, private=False)

print(f"\n✅ Model uploaded to: https://huggingface.co/{OUTPUT_MODEL}")

## 10. Test the Model

In [ ]:
# Re-enable cache for inference
merged_model.config.use_cache = True

# Test prompts
test_prompts = [
    "வணக்கம்",
    "தமிழ்நாட்டின் தலைநகரம் என்ன?",
    "2+2 என்ன?",
    "பொங்கல் எப்போது கொண்டாடப்படுகிறது?",
    "திருக்குறளின் முதல் அதிகாரம் என்ன?",
]

print("\n🧪 Testing model...\n")
print("   (Using anti-repeat decoding per GPT5.2)")
print("   repetition_penalty=1.3, no_repeat_ngram_size=3, top_p=0.9, temp=0.5\n")

for prompt in test_prompts:
    full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
    
    inputs = tokenizer(full_prompt, return_tensors="pt").to(merged_model.device)
    
    with torch.no_grad():
        outputs = merged_model.generate(
            **inputs,
            max_new_tokens=100,
            # Anti-repeat decoding defaults (per GPT5.2)
            temperature=0.5,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Extract assistant response
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1]
        response = response.split("<|im_end|>")[0].strip()
    
    # Check for garbage output
    if "systemsystem" in response.lower() or len(set(response.split())) < 3:
        print(f"Q: {prompt}")
        print(f"A: ❌ GARBAGE/REPEAT DETECTED: {response[:100]}...")
    else:
        print(f"Q: {prompt}")
        print(f"A: {response}")
    print("-" * 50)

## Summary

### Fixes from v3.1:
1. ✅ **Data format consistency** - Only ChatML samples used (raw text excluded)
2. ✅ **Pinned versions** - transformers==4.46.3, trl==0.11.4, etc.
3. ✅ **Single GPU forced** - CUDA_VISIBLE_DEVICES=0 at top
4. ✅ **fp16 on T4** - Not bf16
5. ✅ **4-bit QLoRA** - More stable on Kaggle

### Expected Results:
- No more "systemsystemsystem..." garbage output
- Model should respond coherently in Tamil
- Thirukkural distribution ~25% (down from 71%)